<a href="https://colab.research.google.com/github/diaboloshogunate/GoogleColabML/blob/main/fashionMNIST-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import sys
import torch
from torch import nn
import torchvision
from torchvision.transforms import ToTensor

In [ ]:
# Enable GPU if avialable
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, stack):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = stack

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
# Define train/test
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# datasets
trainset = torchvision.datasets.FashionMNIST('./data', download=True, train=True, transform=ToTensor())
testset = torchvision.datasets.FashionMNIST('./data', download=True, train=False, transform=ToTensor())

In [ ]:
# parameters
for batch_size in [1, 10, 1000]:
    for learning_rate in [1.0, 0.1, 0.01, 0.001]:
        for activation in [nn.ReLU(), nn.Sigmoid()]:
            loss_fn = nn.CrossEntropyLoss()
            layers = [nn.Linear(28*28, 1024), activation, nn.Linear(1024, 1024), activation, nn.Linear(1024, 10)]

            # data loaders
            trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
            testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

            # model
            model = NeuralNetwork(nn.Sequential(*layers)).to(device)
            optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0)

            print(f"Batch: {batch_size}, Learning Rate: {learning_rate}, Activation: {activation}\n")

            # epics
            for epoch in range(2):
                print(f"Epoch {epoch+1}\n-------------------------------")
                train(trainloader, model, loss_fn, optimizer)
                test(testloader, model, loss_fn)

Batch: 1, Learning Rate: 1.0, Activation: ReLU()

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 10.0%, Avg loss:      nan 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 10.0%, Avg loss:      nan 

Batch: 1, Learning Rate: 1.0, Activation: Sigmoid()

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.871818 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.848162 

Batch: 1, Learning Rate: 0.1, Activation: ReLU()

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.313606 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.346536 

Batch: 1, Learning Rate: 0.1, Activation: Sigmoid()

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 76.0%, Avg loss: 0.694039 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 82.4%, Avg loss: 0.474819 

Batch: 1, Learning Rate: 0.01, Activation: ReLU()

Epoch 1

In [7]:
# create table
import pandas as pd

data = [
        [1, 1.0, 'ReLU', 10.0],
        [1, 1.0, 'Sigmoid', 10.0],
        [1, 0.1, 'ReLU', 10.0],
        [1, 0.1, 'Sigmoid', 82.4],
        [1, 0.01, 'ReLU', 84.3],
        [1, 0.01, 'Sigmoid', 81.2],
        [1, 0.001, 'ReLU', 84.2],
        [1, 0.001, 'Sigmoid', 70.3],
  
        [10, 1.0, 'ReLU', 10.0],
        [10, 1.0, 'Sigmoid', 84.4],
        [10, 0.1, 'ReLU', 86.2],
        [10, 0.1, 'Sigmoid', 79.9],
        [10, 0.01, 'ReLU', 83.7],
        [10, 0.01, 'Sigmoid', 71.5],
        [10, 0.001, 'ReLU', 75.6],
        [10, 0.001, 'Sigmoid', 16.2],

        [1000, 1.0, 'ReLU', 10.0],
        [1000, 1.0, 'Sigmoid', 20.9],
        [1000, 0.1, 'ReLU', 71.8],
        [1000, 0.1, 'Sigmoid', 15.1],
        [1000, 0.01, 'ReLU', 54.5],
        [1000, 0.01, 'Sigmoid', 10.0],
        [1000, 0.001, 'ReLU', 17.1],
        [1000, 0.001, 'Sigmoid', 9.3]
]
df = pd.DataFrame(data, [*range(1, 25, 1)], ['Batch Size', 'Learning Rate', 'Activation', 'Accuracy'])
df

,Batch Size,Learning Rate,Activation,Accuracy
1,1,1.000,ReLU,10.0
2,1,1.000,Sigmoid,10.0
3,1,0.100,ReLU,10.0
4,1,0.100,Sigmoid,82.4
5,1,0.010,ReLU,84.3
6,1,0.010,Sigmoid,81.2
7,1,0.001,ReLU,84.2
8,1,0.001,Sigmoid,70.3
9,10,1.000,ReLU,10.0
10,10,1.000,Sigmoid,84.4
